# Importing the necessary modules

In [5]:
import os
import time
import pickle
import numpy as np
import tensorflow as tf
from numpy.linalg import norm
from tqdm import tqdm, tqdm_notebook
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image

# Calling the model

In [6]:
model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

Metal device set to: Apple M2 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

94765736/94765736 [==============================] - 32s 0us/step


In [7]:
def extract_features(img_path, model):
    input_shape = (224, 224, 3)
    img = image.load_img(img_path, target_size=(input_shape[0], input_shape[1]))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    features = model.predict(preprocessed_img)
    flattened_features = features.flatten()
    normalized_features = flattened_features / norm(flattened_features)
    return normalized_features

In [8]:
features = extract_features('caltech-101/101_ObjectCategories/chair/image_0001.jpg', model)
print(len(features))

2023-05-31 23:31:34.127237: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 1s 1s/step
100352


In [9]:
extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG'] 
def get_file_list(root_dir):
    file_list = []
    counter = 1
    for root, directories, filenames in os.walk(root_dir):
        for filename in filenames:
            if any(ext in filename for ext in extensions):
                file_list.append(os.path.join(root, filename))
                counter += 1 
    return file_list

In [10]:
# path to the datasets
root_dir = 'caltech-101/101_ObjectCategories'
filenames = sorted(get_file_list(root_dir))

In [12]:
with tf.device('/GPU:0'):
    feature_list = []
    for i in tqdm_notebook(range(len(filenames))):
        feature_list.append(extract_features(filenames[i], model))

/var/folders/x6/f9t0_d2d1v12tgmn5dhptygc0000gn/T/ipykernel_26637/3850692317.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(filenames))):


  0%|          | 0/8677 [00:00<?, ?it/s]

1/1 [==============================] - 0s 23ms/step


In [ ]:
pickle.dump(feature_list, open('data/features-caltech101-resnet.pickle', 'wb'))
pickle.dump(filenames, open('data/filenames-caltech101.pickle','wb'))